In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


############## Settings for Scotese Paleomap
PaleomapDictionary = {}
PaleomapDictionary['name'] = 'Paleomap'
PaleomapDictionary['reconstruction_model'] = Reconstructions.fetch_Scotese()
PaleomapDictionary['raster_sequence'] = Paleogeography.fetch_Paleomap()
PaleomapDictionary['maximum_time'] = 540.
PaleomapDictionary['time_bin_size'] = 20.
PaleomapDictionary['anchor_plate_id'] = 0
PaleomapDictionary['raster_anchor_plate_id'] = 0

########## Geochemistry Inputs
#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv')
#df = joy.geochem_from_csv('../datafiles/geochem_global_20230124.csv',
#                          longitude_field_name='longitude', latitude_field_name='latitude')

model_dir = '../../andes_paper/luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)



In [ ]:
df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']), crs='EPSG:4326')

'''
df = joymap.select_orogens(df,gdf=None, 
                           orogen_names=['Cordilleran', 
                                         'Terra Australis', 
                                         'Solonker', 
                                         'Indosinian', 
                                         'Variscan-Hercynian',
                                         'Scotian',
                                         'Uralian',
                                         'Mongol',
                                         'Kazakh',
                                         'Caledonia',
                                         'Alpine-Himalayan'], 
                           #continent_names='South America',
                           #region=[-100, -50, -60, 20]
                          )
'''

sz_distance_limit = 750.

df = df.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)

df.plot()

In [ ]:
###########################################
MODEL = PaleomapDictionary
###########################################

region = [-180, 180, -89, 89]
#region = [-76, -66, -25, -12]
projection="Q20c"
perspective = [210, 35]

column_marker_size = '0.1c'
###########################################



time_bin_size = MODEL['time_bin_size']
space_bin_size = 2.

#calibration = 'Hu'
#mohometer_selection = ['la_yb_elevation']

calibration = 'luffi'
mohometer_selection = 41

anchor_plate_id = MODEL['anchor_plate_id']
raster_anchor_plate_id = MODEL['raster_anchor_plate_id']

reconstruction_model = MODEL['reconstruction_model']
raster_sequence = MODEL['raster_sequence']


label_x = 
colorbar_position='JBL+jBL+o14c/1.5c+w8c/0.8c+h'


if isinstance(mohometer_selection, list):
    mohometer_description_string = '|'.join(mohometer_selection)
else:
    mohometer_description_string = str(mohometer_selection)

    
    
for reconstruction_time in np.arange(0,MODEL['maximum_time']+time_bin_size,time_bin_size):
    
    fig = pygmt.Figure()

    binned_elevations = joymap.timeslice_plot(df, reconstruction_time,
                                              time_bin_size, space_bin_size, 
                                              fig, reconstruction_model, raster_sequence,  
                                              anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                              gc_interpolator_dict=gc_interpolator_dict, 
                                              calibration=calibration, mohometer_selection=mohometer_selection,
                                              region=region, projection=projection, perspective=perspective, 
                                              column_marker_size=column_marker_size, coastlines=False,
                                              volcanics=None, return_type='binned_elevations')
    
    joymap.add_labels(fig, reconstruction_time, 
                      add_colorbar=True, colorbar_position=colorbar_position)
    
    fig.savefig(
        '../images/sequence_{:s}/elevations_{:s}_{:s}_{:0.0f}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                    calibration,
                                                                                    mohometer_description_string,
                                                                                    space_bin_size,
                                                                                    reconstruction_time))
    #fig.show(width=1000)
    
    fig = pygmt.Figure()

    binned_residuals = joymap.timeslice_plot(df, reconstruction_time,
                                             time_bin_size, space_bin_size, 
                                             fig, reconstruction_model, raster_sequence,
                                             anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                             gc_interpolator_dict=gc_interpolator_dict, 
                                             calibration=calibration, mohometer_selection=mohometer_selection,
                                             region=region, projection=projection, perspective=perspective, 
                                             column_marker_size=column_marker_size, coastlines=False,
                                             volcanics=None, residuals=True, return_type='binned_residuals')
    
    joymap.add_labels(fig, reconstruction_time, 
                      add_colorbar=True, colorbar_position=colorbar_position,
                      colorbar_title='Residual Elevation [m]')
    
    fig.savefig(
        '../images/sequence_{:s}/elevation_residuals_{:s}_{:s}_{:0.0f}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                             calibration,
                                                                                             mohometer_description_string, 
                                                                                             space_bin_size,
                                                                                             reconstruction_time))
    #fig.show(width=1000)
    
    joymap.residuals_crossplot(
        binned_elevations, 
        binned_residuals,
        fname='../images/sequence_{:s}/crossplot_{:s}_{:s}_{:0.0f}_{:0.1f}Ma.png'.format(MODEL['name'],
                                                                                         calibration,
                                                                                         mohometer_description_string, 
                                                                                         space_bin_size,
                                                                                         reconstruction_time))
    break
    
